# Setup

## Config

In [ ]:
config = {
    "calc_obstat_prob": 0.01,
    "episodes_per_batch": 15,
    "eval_prob": 0.003,
    "l2coeff": 0.005,
    "noise_stdev": 0.02,
    "snapshot_freq": 1,
    "timesteps_per_batch": 100000,
    "return_proc_mode": "centered_rank",
    "episode_cutoff_mode": "env_default"
}

## Environment

In [ ]:
import gym, roboschool # Roboschool import needed to register the environments within gym
env = gym.make("RoboschoolInvertedPendulum-v1")

## Tensorflow Session

In [ ]:
import tensorflow as tf
sess = tf.Session()

## Policy setup

Currently saves the arguments as local variable, then creates a TensorFlow variable scope where the neural network
architecture gets created.

Currently emitted:
1. Observation normalization
2. Obseration clipping
3. _act function
4. _setfromflat
5. _getfromflat
6. set_all_vars

In [ ]:
import numpy as np

args = {
      "ac_bins": "continuous:",
      "ac_noise_std": 0.01,
      #"connection_type": "ff",
      "hidden_dims": [
        256,
        256
      ],
      "nonlin_type": "tanh"
}

ob_space= env.observation_space
ac_space = env.action_space
ac_bins = args["ac_bins"]
ac_noise_std = args["ac_noise_std"]
hidden_dims = args["hidden_dims"]
nonlin = args["nonlin_type"]

with tf.variable_scope("RoboschoolPolicy") as scope:
    # Observation normalization
    #ob_mean = tf.get_variable(
    #    'ob_mean', ob_space.shape, tf.float32, tf.constant_initializer(np.nan), trainable=False)
    #ob_std = tf.get_variable(
    #    'ob_std', ob_space.shape, tf.float32, tf.constant_initializer(np.nan), trainable=False)
    #in_mean = tf.placeholder(tf.float32, ob_space.shape)
    #in_std = tf.placeholder(tf.float32, ob_space.shape)
    #self._set_ob_mean_std = U.function([in_mean, in_std], [], updates=[
        #tf.assign(ob_mean, in_mean),
        #tf.assign(ob_std, in_std),
    #])

    # Policy network

    # Create a placeholder of type float32 with dimension None and the shape of the observation space
    o = tf.placeholder(tf.float32, [None] + list(ob_space.shape))

    # Normalize observation space and clip to [-5.0, 5.0]
    #o = tf.clip_by_value((o - ob_mean) / ob_std, -5.0, 5.0)


    # Feed-Forward Neural Network architecture
    x = o
    # Iterate through the hidden dimensions. In each iteration create a dense layer and activate it with the
    # self.nonlin activation function
    for ilayer, hd in enumerate(hidden_dims):
        shape = [x.get_shape()[1], hd]
        std = 1.0
        
        # Initializer for the newly created weights. TODO possible replacement tf.keras.initializers.RandomNormal
        out = np.random.randn(*shape).astype(np.float32)
        out *= std / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
        initializer= tf.constant(out)
        
        # Creates a Dense layer TODO possible replacement tf.keras.layers.Dense
        w = tf.get_variable("l" + str(ilayer) + "/w", shape, initializer=initializer)
        b = tf.get_variable("l" + str(ilayer) + "/b", [hd], initializer=tf.zeros_initializer)
        dense = tf.matmul(x, w) + b
       
        x = nonlin(dense)


    # Map to action
    adim = ac_space.shape[0]
    
    # Initializer for the newly created weights. TODO possible replacement tf.keras.initializers.RandomNormal
    out = np.random.randn(*adim).astype(np.float32)
    out *=  0.01 / np.sqrt(np.square(out).sum(axis=0, keepdims=True))
    initializer= tf.constant(out)
    
    # Creates a Dense layer TODO possible replacement tf.keras.layers.Dense
    w = tf.get_variable("out" + "/w", [x.get_shape()[1], adim], initializer=initializer)
    b = tf.get_variable("out" + "/b", [adim], initializer=tf.zeros_initializer)
    dense = tf.matmul(x, w) + b
    
    a = dense
    
    # TODO _act


all_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope.name)
trainable_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope.name)

#self.num_params = sum(int(np.prod(v.get_shape().as_list())) for v in self.trainable_variables)

#self._setfromflat = U.SetFromFlat(self.trainable_variables)
#self._getflat = U.GetFlat(self.trainable_variables)

#placeholders = [tf.placeholder(v.value().dtype, v.get_shape().as_list()) for v in self.all_variables]

# self.set_all_vars = U.function(
#     inputs=placeholders,
#     outputs=[],
#     updates=[tf.group(*[v.assign(p) for v, p in zip(self.all_variables, placeholders)])]
# )

In [ ]:
#optimizer = {'sgd': SGD, 'adam': Adam}[exp['optimizer']['type']](policy, **exp['optimizer']['args'])

## Shared Memory

In [ ]:
import ctypes, multiprocessing

seed = 123
count = 250000000  # 1 gigabyte of 32-bit numbers. Will actually sample 2 gigabytes below.

# Instantiate an array of C float datatype with size count
_shared_mem = multiprocessing.Array(ctypes.c_float, count)

# Convert to numpy array
_noise = np.ctypeslib.as_array(_shared_mem.get_obj())
assert _noise.dtype == np.float32
_noise[:] = np.random.RandomState(seed).randn(count)  # 64-bit to 32-bit conversion here

# def get(self, i, dim):
#     return self.noise[i:i + dim]
# 
# def sample_index(self, stream, dim):
#     return stream.randint(0, len(self.noise) - dim + 1)

# noise = SharedNoiseTable()

In [ ]:
import time

rs = np.random.RandomState()


# ob_stat = RunningStat(
#     env.observation_space.shape,
#     eps=1e-2  # eps to prevent dividing by zero at the beginning when computing mean/stdev
# )



tslimit, incr_tslimit_threshold, tslimit_incr_ratio = None, None, None
adaptive_tslimit = False


episodes_so_far = 0
timesteps_so_far = 0
tstart = time.time()

task_counter = 0

while True:
    step_tstart = time.time()
    theta = policy.get_trainable_flat()
    assert theta.dtype == np.float32

    # Task counter is used to recognize false tasks from previous iterations later
    curr_task_id = task_counter
    task_counter += 1

    tasks.append(Task(
            params=theta,
            ob_mean=ob_stat.mean if policy.needs_ob_stat else None,
            ob_std=ob_stat.std if policy.needs_ob_stat else None,
            timestep_limit=tslimit,
            task_id = curr_task_id
    ))

    tlogger.log('********** Iteration {} **********'.format(curr_task_id))

    # Pop off results for the current task
    curr_task_results, eval_rets, eval_lens, worker_ids = [], [], [], []
    num_results_skipped, num_episodes_popped, num_timesteps_popped, ob_count_this_batch = 0, 0, 0, 0
    while num_episodes_popped < config.episodes_per_batch:
        # Wait for a result
        result = pop_item(result_queue, lock)

        assert isinstance(result, Result)
        task_id = result.task_id
        assert isinstance(task_id, int)

        assert (result.eval_return is None) == (result.eval_length is None)
        worker_ids.append(result.worker_id)

        if result.eval_length is not None:
            # This was an eval job
            episodes_so_far += 1
            timesteps_so_far += result.eval_length
            # Store the result only for current tasks
            if task_id == curr_task_id:
                eval_rets.append(result.eval_return)
                eval_lens.append(result.eval_length)
        else:
            # The real shit
            assert (result.noise_inds_n.ndim == 1 and
                    result.returns_n2.shape == result.lengths_n2.shape == (len(result.noise_inds_n), 2))
            assert result.returns_n2.dtype == np.float32
            # Update counts
            result_num_eps = result.lengths_n2.size
            result_num_timesteps = result.lengths_n2.sum()
            episodes_so_far += result_num_eps
            timesteps_so_far += result_num_timesteps
            # Store results only for current tasks
            if task_id == curr_task_id:
                curr_task_results.append(result)
                num_episodes_popped += result_num_eps
                num_timesteps_popped += result_num_timesteps
                # Update ob stats
                if policy.needs_ob_stat and result.ob_count > 0:
                    ob_stat.increment(result.ob_sum, result.ob_sumsq, result.ob_count)
                    ob_count_this_batch += result.ob_count
            else:
                num_results_skipped += 1

    # Compute skip fraction
    frac_results_skipped = num_results_skipped / (num_results_skipped + len(curr_task_results))
    if num_results_skipped > 0:
        logger.warning('Skipped {} out of date results ({:.2f}%)'.format(
            num_results_skipped, 100. * frac_results_skipped))

    # Assemble results
    noise_inds_n = np.concatenate([r.noise_inds_n for r in curr_task_results])
    returns_n2 = np.concatenate([r.returns_n2 for r in curr_task_results])
    lengths_n2 = np.concatenate([r.lengths_n2 for r in curr_task_results])
    assert noise_inds_n.shape[0] == returns_n2.shape[0] == lengths_n2.shape[0]
    # Process returns
    if config.return_proc_mode == 'centered_rank':
        proc_returns_n2 = compute_centered_ranks(returns_n2)
    elif config.return_proc_mode == 'sign':
        proc_returns_n2 = np.concatenate([r.signreturns_n2 for r in curr_task_results])
    elif config.return_proc_mode == 'centered_sign_rank':
        proc_returns_n2 = compute_centered_ranks(np.concatenate([r.signreturns_n2 for r in curr_task_results]))
    else:
        raise NotImplementedError(config.return_proc_mode)
    # Compute and take step
    g, count = batched_weighted_sum(
        proc_returns_n2[:, 0] - proc_returns_n2[:, 1],
        (noise.get(idx, policy.num_params) for idx in noise_inds_n),
        batch_size=500
    )
    g /= returns_n2.size
    assert g.shape == (policy.num_params,) and g.dtype == np.float32 and count == len(noise_inds_n)
    #update_ratio = optimizer.update(-g + config.l2coeff * theta)
    update_ratio = optimizer.update(config.l2coeff * g)

    # Update ob stat (we're never running the policy in the master, but we might be snapshotting the policy)
    if policy.needs_ob_stat:
        policy.set_ob_stat(ob_stat.mean, ob_stat.std)

    # Update number of steps to take
    if adaptive_tslimit and (lengths_n2 == tslimit).mean() >= incr_tslimit_threshold:
        old_tslimit = tslimit
        tslimit = int(tslimit_incr_ratio * tslimit)
        logger.info('Increased timestep limit from {} to {}'.format(old_tslimit, tslimit))

    step_tend = time.time()
    tlogger.record_tabular("EpRewMean", returns_n2.mean())
    tlogger.record_tabular("EpRewStd", returns_n2.std())
    tlogger.record_tabular("EpLenMean", lengths_n2.mean())

    tlogger.record_tabular("EvalEpRewMean", np.nan if not eval_rets else np.mean(eval_rets))
    tlogger.record_tabular("EvalEpRewStd", np.nan if not eval_rets else np.std(eval_rets))
    tlogger.record_tabular("EvalEpLenMean", np.nan if not eval_rets else np.mean(eval_lens))
    tlogger.record_tabular("EvalPopRank", np.nan if not eval_rets else (
        np.searchsorted(np.sort(returns_n2.ravel()), eval_rets).mean() / returns_n2.size))
    tlogger.record_tabular("EvalEpCount", len(eval_rets))

    tlogger.record_tabular("Norm", float(np.square(policy.get_trainable_flat()).sum()))
    tlogger.record_tabular("GradNorm", float(np.square(g).sum()))
    tlogger.record_tabular("UpdateRatio", float(update_ratio))

    tlogger.record_tabular("EpisodesThisIter", lengths_n2.size)
    tlogger.record_tabular("EpisodesSoFar", episodes_so_far)
    tlogger.record_tabular("TimestepsThisIter", lengths_n2.sum())
    tlogger.record_tabular("TimestepsSoFar", timesteps_so_far)

    num_unique_workers = len(set(worker_ids))
    tlogger.record_tabular("UniqueWorkers", num_unique_workers)
    tlogger.record_tabular("UniqueWorkersFrac", num_unique_workers / len(worker_ids))
    tlogger.record_tabular("ResultsSkippedFrac", frac_results_skipped)
    tlogger.record_tabular("ObCount", ob_count_this_batch)

    tlogger.record_tabular("TimeElapsedThisIter", step_tend - step_tstart)
    tlogger.record_tabular("TimeElapsed", step_tend - tstart)
    tlogger.dump_tabular()

    if config.snapshot_freq != 0 and curr_task_id % config.snapshot_freq == 0:
        import os.path as osp
        filename = osp.join(tlogger.get_dir(), 'snapshot_iter{:05d}_rew{}.h5'.format(
            curr_task_id,
            np.nan if not eval_rets else int(np.mean(eval_rets))
        ))
        assert not osp.exists(filename)
        policy.save(filename)
        tlogger.log('Saved snapshot {}'.format(filename))
